# BirdCLEF+ 2025: CRNN Audio Model
This notebook implements the Convolutional Recurrent Neural Network (CRNN) for the BirdCLEF+ 2025 competition, by using a pretrained CNN model from `cnn_mel_spectrogram.ipynb` and adding a recurrent layer to capture temporal dynamics.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
import os

# Define paths for Colab
DATA_PATH = '/content/drive/MyDrive/birdclef-2025-data'
MODEL_SAVE_DIR = '/content/drive/MyDrive/fp-561-models'
CNN_MODEL_PATH = '/content/drive/MyDrive/fp-561-models/cnn_model.pth'  # Path to saved CNN model

# Create directory if it doesn't exist
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Install required packages
!pip install -q librosa torchlibrosa timm torchaudio

# Imports
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import librosa
import librosa.display
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import copy

In [ ]:
# Reproducibility and device setup
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed()
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')

## Data Loading
Load training metadata and define paths.

In [ ]:
# Verify data path
print(f'Data path exists: {os.path.exists(DATA_PATH)}')
if not os.path.exists(DATA_PATH):
    print('Please check that you have uploaded the birdclef-2025-data folder to your Google Drive')

# Load training data
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
print(f'Training samples: {len(train_df)}')
train_df.head()

## Feature Extraction
Define mel-spectrogram extraction functions.

In [ ]:
def compute_melspec(audio, sr=32000, n_mels=128, fmin=20, fmax=16000, n_fft=1024, hop_length=512):
    """Compute and normalize mel-spectrogram"""
    S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, fmin=fmin, fmax=fmax, n_fft=n_fft, hop_length=hop_length, power=2.0)
    S_db = librosa.power_to_db(S, ref=np.max)
    norm = (S_db - S_db.min()) / (S_db.max() - S_db.min() + 1e-6)
    return norm

def audio_to_melspec(audio, sr=32000, duration=5):
    """Convert audio to fixed-length mel-spectrogram"""
    target_len = int(sr * duration)
    if len(audio) > target_len:
        # Random crop if audio is longer than target
        start = np.random.randint(0, len(audio)-target_len)
        audio = audio[start:start+target_len]
    else:
        # Pad with zeros if audio is shorter than target
        pad = target_len - len(audio)
        audio = np.pad(audio, (pad//2, pad-pad//2))
    melspec = compute_melspec(audio, sr=sr)
    return melspec[np.newaxis, :, :]  # add channel dimension

## Audio Augmentation
Define augmentation functions for better generalization.

In [ ]:
def time_shift(audio, shift_factor=0.2):
    """Apply random time shift to audio"""
    shift = int(len(audio) * shift_factor)
    direction = np.random.randint(0, 2)
    if direction == 1:
        shift = -shift
    aug_audio = np.roll(audio, shift)
    # Set the rolled part to zero
    if shift > 0:
        aug_audio[:shift] = 0
    else:
        aug_audio[shift:] = 0
    return aug_audio

def add_noise(audio, noise_factor=0.01):
    """Add random Gaussian noise to audio"""
    noise = np.random.normal(0, audio.std() * noise_factor, audio.shape)
    return audio + noise

def pitch_shift(audio, sr, pitch_factor=2):
    """Change pitch of audio without changing tempo"""
    pitch_change = np.random.randint(-pitch_factor, pitch_factor)
    return librosa.effects.pitch_shift(audio, sr=sr, n_steps=pitch_change)

def apply_augmentation(audio, sr=32000):
    """Apply a random combination of augmentations"""
    # List of possible augmentations
    augmentations = [
        lambda x: time_shift(x),
        lambda x: add_noise(x),
        lambda x: pitch_shift(x, sr),
        lambda x: x  # No augmentation
    ]
    
    # Randomly select number of augmentations to apply
    n_augments = np.random.randint(0, 3)  # 0, 1, or 2 augmentations
    aug_indices = np.random.choice(len(augmentations), size=n_augments, replace=False)
    
    # Apply selected augmentations
    result = audio
    for idx in aug_indices:
        result = augmentations[idx](result)
    
    return result

## Dataset and DataLoader
Define PyTorch dataset for CRNN.

In [ ]:
class CRNNDataset(Dataset):
    def __init__(self, df, path, sr=32000, duration=5, augment=False):
        self.df = df.reset_index(drop=True)
        self.path = path
        self.sr = sr
        self.duration = duration
        self.augment = augment
        self.labels = sorted(df['primary_label'].unique())
        self.label2idx = {l:i for i,l in enumerate(self.labels)}
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, i):
        # Get file path and load audio
        row = self.df.iloc[i]
        audio, _ = sf.read(os.path.join(self.path, 'train_audio', row.filename))
        
        # Apply augmentation if enabled
        if self.augment:
            audio = apply_augmentation(audio, self.sr)
        
        # Convert to mel-spectrogram
        melspec = audio_to_melspec(audio, sr=self.sr, duration=self.duration)
        
        # Get label index
        label = self.label2idx[row.primary_label]
        
        return torch.tensor(melspec, dtype=torch.float32), torch.tensor(label)

## CRNN Model Definition Using Pretrained CNN
Define a new CRNN architecture that uses the pretrained CNN as a feature extractor and adds a GRU layer.

In [ ]:
class CNNFeatureExtractor(nn.Module):
    """Feature extractor based on the pre-trained CNN model"""
    def __init__(self, cnn_model_info):
        super().__init__()
        self.model_type = cnn_model_info.get('model_type', 'unknown')
        
        # Import the CNN model architecture from the cnn_mel_spectrogram notebook
        # This is a simplified version - adjust based on actual CNN architecture
        if 'backbone' in cnn_model_info and cnn_model_info['backbone']:
            # It's a pretrained model using timm
            import timm
            backbone = cnn_model_info['backbone']
            self.cnn = timm.create_model(
                backbone,
                pretrained=False,
                in_chans=1,  # Single channel for grayscale mel-spectrogram
                num_classes=0  # No classification head
            )
            
            # Handle specific architecture types
            if 'efficientnet' in backbone:
                self.feature_dim = self.cnn.num_features
            elif 'resnet' in backbone:
                self.feature_dim = self.cnn.fc.in_features
                self.cnn.fc = nn.Identity()
            else:
                # Default assumption
                self.feature_dim = 512
        else:
            # Assume it's a custom CNN architecture
            self.cnn = nn.Sequential(
                nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
                nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2),
                nn.AdaptiveAvgPool2d(1),  # Global average pooling
                nn.Flatten()
            )
            self.feature_dim = 256
        
        # Load the pretrained weights
        try:
            # Try to load the CNN part only (exclude classifier weights if needed)
            model_state = cnn_model_info['model_state']
            
            # Filter out classifier weights if present
            cnn_state_dict = {k: v for k, v in model_state.items() 
                             if not k.startswith('classifier') and not k.startswith('fc')}
            
            # Load the weights into the CNN part
            missing, unexpected = self.cnn.load_state_dict(cnn_state_dict, strict=False)
            print(f"Loaded CNN weights: {len(cnn_state_dict)} parameters")
            print(f"Missing: {len(missing)}, Unexpected: {len(unexpected)}")
        except Exception as e:
            print(f"Error loading CNN weights: {e}")
            print("Continuing with randomly initialized weights")
        
        print(f"CNN Feature extractor initialized with feature dimension: {self.feature_dim}")

    def forward(self, x):
        features = self.cnn(x)  # B, feature_dim
        return features


class CNNGRUModel(nn.Module):
    """CRNN model using pretrained CNN with GRU layer added"""
    def __init__(self, cnn_model_info, num_classes, gru_hidden_size=256, freeze_cnn=True):
        super().__init__()
        
        # CNN feature extractor from pretrained model
        self.cnn = CNNFeatureExtractor(cnn_model_info)
        
        # Freeze CNN weights if required
        if freeze_cnn:
            for param in self.cnn.parameters():
                param.requires_grad = False
            print("CNN weights frozen")
        
        # GRU layer
        self.gru_hidden_size = gru_hidden_size
        self.gru = nn.GRU(
            input_size=self.cnn.feature_dim,
            hidden_size=gru_hidden_size,
            batch_first=True,
            bidirectional=True,
            num_layers=2,
            dropout=0.3
        )
        
        # Output layer
        self.dropout = nn.Dropout(0.5)
        self.classifier = nn.Linear(gru_hidden_size * 2, num_classes)  # *2 for bidirectional
    
    def forward(self, x):
        batch_size = x.size(0)
        
        # We need time slices for GRU
        # Split spectrogram into overlapping windows along time dimension
        # Assuming x shape: [batch_size, 1, freq_bins, time_steps]
        time_slices = []
        window_size = 16
        stride = 8
        
        if x.size(3) < window_size:  # Handle short spectrograms
            # Pass the whole spectrogram through CNN
            features = self.cnn(x)  # [batch_size, feature_dim]
            # Add time dimension
            features = features.unsqueeze(1)  # [batch_size, 1, feature_dim]
        else:
            # Split into time windows and extract features from each
            features_list = []
            
            for i in range(0, x.size(3) - window_size + 1, stride):
                # Extract window
                window = x[:, :, :, i:i+window_size]  # [batch_size, 1, freq_bins, window_size]
                # Extract features from this window
                feat = self.cnn(window)  # [batch_size, feature_dim]
                features_list.append(feat)
            
            # Stack along time dimension
            if features_list:
                features = torch.stack(features_list, dim=1)  # [batch_size, num_windows, feature_dim]
            else:
                # Fallback for very short spectrograms
                features = self.cnn(x).unsqueeze(1)  # [batch_size, 1, feature_dim]
        
        # Apply GRU to the sequence of CNN features
        gru_out, _ = self.gru(features)  # [batch_size, seq_len, 2*hidden_size]
        
        # Use the final output for classification
        out = gru_out[:, -1, :]  # [batch_size, 2*hidden_size]
        out = self.dropout(out)
        logits = self.classifier(out)
        
        return logits

## Loading the Pretrained CNN Model
Load the CNN model that was trained on mel spectrograms.

In [ ]:
def load_pretrained_cnn():
    """Load the pretrained CNN model from the saved file"""
    if not os.path.exists(CNN_MODEL_PATH):
        print(f"Error: CNN model not found at {CNN_MODEL_PATH}")
        print("Please check that you have trained and saved the CNN model from cnn_mel_spectrogram.ipynb")
        return None
    
    try:
        # Load the model info
        cnn_model_info = torch.load(CNN_MODEL_PATH, map_location=DEVICE)
        print(f"Loaded CNN model info from {CNN_MODEL_PATH}")
        
        # Check what type of model info we have
        if isinstance(cnn_model_info, dict) and 'model_state' in cnn_model_info:
            # This is the standardized format with metadata
            print(f"Model type: {cnn_model_info.get('model_type', 'unknown')}")
            model_state = cnn_model_info['model_state']
            label_mapping = cnn_model_info.get('label_mapping', None)
            
            # Additional useful info if available
            if 'input_params' in cnn_model_info:
                print(f"Input parameters: {cnn_model_info['input_params']}")
            if 'backbone' in cnn_model_info and cnn_model_info['backbone']:
                print(f"Backbone: {cnn_model_info['backbone']}")
                
            return cnn_model_info
                
        elif isinstance(cnn_model_info, dict) and 'model_state_dict' in cnn_model_info:
            # This is a checkpoint format with state_dict
            print("Loaded model in checkpoint format")
            model_state = cnn_model_info['model_state_dict']
            label_mapping = cnn_model_info.get('label_mapping', None)
            
            # Create a standardized format
            return {
                'model_state': model_state,
                'label_mapping': label_mapping
            }
            
        elif isinstance(cnn_model_info, dict) and any(k.endswith('.weight') for k in cnn_model_info.keys()):
            # This is a raw state_dict
            print("Loaded raw model state dict")
            return {
                'model_state': cnn_model_info,
                'label_mapping': None
            }
            
        else:
            print("Unknown model format. Attempting to use as state dict...")
            return {
                'model_state': cnn_model_info,
                'label_mapping': None
            }
            
    except Exception as e:
        print(f"Error loading CNN model: {e}")
        return None

# Load the CNN model
cnn_model_info = load_pretrained_cnn()

## Training and Validation
Define training and validation loops.

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for x, y in tqdm(loader):
        x, y = x.to(device), y.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update statistics
        running_loss += loss.item() * x.size(0)
        _, predicted = outputs.max(1)
        total += y.size(0)
        correct += predicted.eq(y).sum().item()
    
    return running_loss / total, correct / total

def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    all_outputs = []
    all_targets = []
    
    with torch.no_grad():
        for x, y in tqdm(loader):
            x, y = x.to(device), y.to(device)
            
            # Forward pass
            outputs = model(x)
            loss = criterion(outputs, y)
            
            # Update statistics
            running_loss += loss.item() * x.size(0)
            _, predicted = outputs.max(1)
            total += y.size(0)
            correct += predicted.eq(y).sum().item()
            
            # Store outputs and targets for AUC calculation
            all_outputs.append(F.softmax(outputs, dim=1).cpu().numpy())
            all_targets.append(F.one_hot(y, num_classes=outputs.size(1)).cpu().numpy())
    
    # Concatenate all outputs and targets
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    
    # Calculate ROC-AUC (which is also the competition metric)
    from sklearn.metrics import roc_auc_score
    pos = (all_targets.sum(0) > 0)  # Classes with positive examples
    auc = roc_auc_score(all_targets[:, pos], all_outputs[:, pos], average='macro')
    
    return running_loss / total, correct / total, auc

## Main Training Routine
Prepare loaders, model, and train.

In [ ]:
def main(freeze_cnn=True):
    # Prepare data
    from sklearn.model_selection import train_test_split
    tr, va = train_test_split(train_df, test_size=0.2, stratify=train_df.primary_label, random_state=42)
    
    # Create datasets
    train_ds = CRNNDataset(tr, DATA_PATH, augment=True)
    val_ds = CRNNDataset(va, DATA_PATH, augment=False)
    
    # Create data loaders
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=64, num_workers=2)
    
    # Number of classes
    num_classes = len(train_ds.labels)
    print(f"Number of classes: {num_classes}")
    
    # Use our pretrained CNN + GRU model
    model = CNNGRUModel(
        cnn_model_info=cnn_model_info,
        num_classes=num_classes,
        gru_hidden_size=256,
        freeze_cnn=freeze_cnn
    ).to(DEVICE)
    
    model_name = "crnn_with_pretrained_cnn"
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer - different learning rates for different parts
    if freeze_cnn:
        # Don't include CNN parameters in optimizer
        optimizer = optim.AdamW([
            {'params': model.gru.parameters(), 'lr': 3e-4},
            {'params': model.classifier.parameters(), 'lr': 3e-4}
        ], weight_decay=1e-4)
    else:
        # Include CNN parameters with lower learning rate
        optimizer = optim.AdamW([
            {'params': model.cnn.parameters(), 'lr': 5e-5},  # Lower LR for CNN
            {'params': model.gru.parameters(), 'lr': 3e-4},  # Higher LR for GRU
            {'params': model.classifier.parameters(), 'lr': 3e-4}  # Higher LR for classifier
        ], weight_decay=1e-4)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    
    # Training parameters
    best_auc = 0
    patience = 5
    wait = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'val_auc': []}
    best_model_path = os.path.join(MODEL_SAVE_DIR, f"{model_name}_best.pt")
    
    # Save label mapping for later use
    label_mapping = train_ds.label2idx
    torch.save(label_mapping, os.path.join(MODEL_SAVE_DIR, f"{model_name}_label_mapping.pt"))
    print(f"Saved label mapping with {len(label_mapping)} classes")
    
    # Training loop
    for epoch in range(20):
        print(f"\nEpoch {epoch+1}/20")
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
        
        # Validate
        val_loss, val_acc, val_auc = validate(model, val_loader, criterion, DEVICE)
        
        # Update scheduler
        scheduler.step(val_auc)
        
        # Print results
        print(f'Epoch {epoch+1}: train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, val_auc={val_auc:.4f}')
        
        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_auc'].append(val_auc)
        
        # Save best model
        if val_auc > best_auc:
            best_auc = val_auc
            torch.save({
                'model_state_dict': model.state_dict(),
                'epoch': epoch,
                'val_auc': val_auc,
                'label_mapping': train_ds.label2idx
            }, best_model_path)
            wait = 0
            print(f"New best model with val_auc={val_auc:.4f}")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    # Load best model
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Training complete. Best validation AUC: {best_auc:.4f} at epoch {checkpoint['epoch']+1}")
    
    # Save final model in a format suitable for ensemble
    torch.save({
        'model_state': model.state_dict(),
        'label_mapping': label_mapping,
        'model_type': 'crnn_with_pretrained_cnn',
        'input_params': {
            'sr': 32000,
            'n_mels': 128,
            'fmin': 20,
            'fmax': 16000,
            'n_fft': 1024,
            'hop_length': 512,
            'duration': 5
        },
        'history': history
    }, os.path.join(MODEL_SAVE_DIR, f"{model_name}_ensemble.pth"))
    
    print(f"Model saved for ensemble use at '{os.path.join(MODEL_SAVE_DIR, f'{model_name}_ensemble.pth')}")
    
    return model, label_mapping, history

## Run Training
Execute the training process with the pretrained CNN + GRU.

In [ ]:
# Choose whether to freeze the CNN weights
FREEZE_CNN = True  # Set to False to fine-tune the CNN as well

# Train model
model, label_mapping, history = main(freeze_cnn=FREEZE_CNN)

## Visualize Training Results
Plot training and validation metrics.

In [ ]:
def plot_history(history):
    plt.figure(figsize=(18, 5))
    
    # Plot training & validation loss
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'], label='Validation')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot training & validation accuracy
    plt.subplot(1, 3, 2)
    plt.plot(history['train_acc'], label='Train')
    plt.plot(history['val_acc'], label='Validation')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot validation AUC
    plt.subplot(1, 3, 3)
    plt.plot(history['val_auc'], label='Validation')
    plt.title('ROC AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot training history
plot_history(history)

## Model Evaluation
Evaluate the model on the validation set with more detailed metrics.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

def evaluate_model(model, dataset, device):
    loader = DataLoader(dataset, batch_size=64, num_workers=2)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device)
            outputs = model(x)
            preds = outputs.argmax(1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y.numpy())
    
    # Convert indices to class names
    idx_to_label = {v: k for k, v in dataset.label2idx.items()}
    class_names = [idx_to_label[i] for i in range(len(idx_to_label))]
    
    # Compute confusion matrix for a subset of classes (top 20 by frequency)
    class_counts = np.bincount([l for l in all_labels if l < len(class_names)])
    top_classes = np.argsort(class_counts)[-20:]  # Top 20 most frequent classes
    
    # Filter predictions and labels for top classes only
    mask = np.isin(all_labels, top_classes)
    filtered_preds = [all_preds[i] for i, m in enumerate(mask) if m]
    filtered_labels = [all_labels[i] for i, m in enumerate(mask) if m]
    
    # Compute and plot confusion matrix
    cm = confusion_matrix(filtered_labels, filtered_preds)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=[class_names[i] for i in top_classes],
                yticklabels=[class_names[i] for i in top_classes])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix (Top 20 Classes)')
    plt.tight_layout()
    plt.show()
    
    # Print classification report
    report = classification_report(
        all_labels, all_preds, 
        target_names=[idx_to_label[i] for i in range(len(idx_to_label))],
        output_dict=True
    )
    
    # Display top and bottom performing classes
    df_report = pd.DataFrame(report).transpose()
    top_performing = df_report.sort_values(by='f1-score', ascending=False).head(10)
    bottom_performing = df_report.sort_values(by='f1-score').head(10)
    
    print("Top 10 best predicted classes:")
    print(top_performing[['precision', 'recall', 'f1-score', 'support']])
    
    print("\nBottom 10 worst predicted classes:")
    print(bottom_performing[['precision', 'recall', 'f1-score', 'support']])

In [ ]:
# Create validation dataset
from sklearn.model_selection import train_test_split
_, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df.primary_label, random_state=42)
val_dataset = CRNNDataset(val_df, DATA_PATH, augment=False)

# Evaluate model
evaluate_model(model, val_dataset, DEVICE)

## Conclusion

We've successfully implemented a CRNN model for the BirdCLEF+ 2025 competition using the already trained CNN model from `cnn_mel_spectrogram.ipynb` as a feature extractor and adding GRU layers on top. This approach:

1. Leverages the feature extraction capabilities already learned by the CNN
2. Adds recurrent layers to capture temporal dynamics in the audio
3. Reduces training time by reusing the pretrained CNN
4. Creates complementary model characteristics for better ensemble results

The model has been saved in Google Drive and is ready to be used as part of an ensemble approach.